In [71]:
import pandas as pd
import numpy as np


In [15]:
car = pd.read_csv('quikr_car.csv')
car = car[car["year"].str.isnumeric()]



In [16]:
car['year']=car['year'].astype(int)

In [17]:
car=car[car['Price']!='Ask For Price']

In [20]:
car['Price']=car['Price'].str.replace(',','').astype(int)

In [26]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [27]:
car = car[car['kms_driven'].str.isnumeric()]

In [29]:
car['kms_driven'] = car['kms_driven'].astype(int)

In [32]:
car = car[~car['fuel_type'].isna()]

In [36]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [40]:
car = car.reset_index(drop=True)

In [45]:
car = car[car['Price']<6e6].reset_index(drop=True)


In [46]:
car.to_csv('Cleaned_car_data.csv')

### MODEL

In [48]:
x=car.drop(columns='Price')
y=car['Price']

In [51]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [53]:
ohe = OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [60]:
column_trans = make_column_transformer((OneHotEncoder(ohe.categories_),['name','company','fuel_type']),
                                       remainder='passthrough')

In [61]:
lr = LinearRegression()


In [64]:
pipe=make_pipeline(column_trans,lr)

In [65]:
pipe.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'B...
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)],
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                          

In [68]:
y_pred=pipe.predict(x_test)
r2_score(y_test,y_pred)

0.748624379226312

In [69]:
scores=[]
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

In [72]:
np.argmax(scores)


655

In [73]:
scores[np.argmax(scores)]

0.920084111126408

In [75]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
r2_score(y_test,y_pred)

0.920084111126408

In [76]:
import pickle

In [77]:
pickle.dump(pipe,open('Linearregressionmodel.pkl','wb'))